In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Create a function that returns possible values for ace(s) in hand
def ace_values_new(num_aces):
    if (num_aces == 0):
        return [0]
    else:
        return[num_aces,10+num_aces]

In [8]:
# Make a deck
def make_decks(num_decks, card_types):
    new_deck = []
    for i in range(num_decks):
        for j in range(4):
            new_deck.extend(card_types)
    random.shuffle(new_deck)
    return new_deck

# Total up value of hand
def total_up(hand):
    aces = 0
    total = 0
    
    for card in hand:
        if card != 'A':
            total += card
        else:
            aces += 1
    
    # Call function ace_values to produce list of possible values for aces in hand
    ace_value_list = ace_values_new(aces)
    final_totals = [i+total for i in ace_value_list if i+total<=21]
    
    # If final_totals has two entries, take the max (means both Ace 1 and 11 do not bust)
    # Otherwise, only Ace 1 does not bust so take the only value of the list
    if len(final_totals) > 1:
        return max(final_totals)
    else:
        return final_totals[0]

array([[0.]])

In [69]:
stacks = 5
players = 1
num_decks = 1

# In blackjack, the face cards and 10s are all the same
# So we have four 10s per deck (10, Jack, Queen, King)
card_types = ['A',2,3,4,5,6,7,8,9,10,10,10,10]

dealer_card_feature = []
player_card_feature = []
player_results = []

blackjack = set(['A',10])

for stack in range(stacks):
    dealer_cards = make_decks(num_decks, card_types)
    while len(dealer_cards) > 20:
        
        curr_player_results = np.zeros(players)
        
        dealer_hand = []
        player_hands = [[] for player in range(players)]

        # Deal FIRST card
        for player in range(players):
            player_hands[player].append(dealer_cards.pop(0))
        dealer_hand.append(dealer_cards.pop(0))
        
        # Deal SECOND card
        for player in range(players):
            player_hands[player].append(dealer_cards.pop(0))
        dealer_hand.append(dealer_cards.pop(0))

        # Dealer checks for 21
        if set(dealer_hand) == blackjack:
            for player in range(players):
                if set(player_hands[player]) != blackjack:
                    curr_player_results[player] = -1
                else:
                    curr_player_results[player] = 0
        else:
            for player in range(players):
                # Players check for 21
                if set(player_hands[player]) == blackjack:
                    curr_player_results[player] = 1
                else:
                    # Hit randomly, check for busts
                    while (random.random() >= 0.5) and (total_up(player_hands[player]) != 21):
                        player_hands[player].append(dealer_cards.pop(0))
                        if total_up(player_hands[player]) > 21:
                            curr_player_results[player] = -1
                            break
        
        # Dealer hits based on the rules
        while total_up(dealer_hand) < 17:
            dealer_hand.append(dealer_cards.pop(0))
        # Compare dealer hand to players hand but first check if dealer busted
        if total_up(dealer_hand) > 21:
            for player in range(players):
                
                # If the player hasn't already busted, they win (since the dealer busted)
                if curr_player_results[player] != -1:
                    curr_player_results[player] = 1
        else:
            for player in range(players):
                
                # If player did not bust
                if curr_player_results[player] != -1:
                    if total_up(player_hands[player]) > total_up(dealer_hand):
                        curr_player_results[player] = 1
                    elif total_up(player_hands[player]) == total_up(dealer_hand):
                        curr_player_results[player] = 0
                    else:
                        continue
            
        # Track features
        dealer_card_feature.append(dealer_hand[0])
        player_card_feature.append(player_hands)
        player_results.append(list(curr_player_results))

'''
model_df = pd.DataFrame()
model_df['dealer_card'] = dealer_card_feature
model_df['player_total_initial'] = [total_up(i[0][0:2]) for i in player_card_feature]
model_df['Y'] = [i[0] for i in player_results]

lose = []
for i in model_df['Y']:
    if i == -1:
        lose.append(1)
    else:
        lose.append(0)
model_df['lose'] = lose

has_ace = []
for i in player_card_feature:
    if ('A' in i[0][0:2]):
        has_ace.append(1)
    else:
        has_ace.append(0)
model_df['has_ace'] = has_ace

dealer_card_num = []
for i in model_df['dealer_card']:
    if i=='A':
        dealer_card_num.append(11)
    else:
        dealer_card_num.append(i)
model_df['dealer_card_num'] = dealer_card_num
'''

"\nmodel_df = pd.DataFrame()\nmodel_df['dealer_card'] = dealer_card_feature\nmodel_df['player_total_initial'] = [total_up(i[0][0:2]) for i in player_card_feature]\nmodel_df['Y'] = [i[0] for i in player_results]\n\nlose = []\nfor i in model_df['Y']:\n    if i == -1:\n        lose.append(1)\n    else:\n        lose.append(0)\nmodel_df['lose'] = lose\n\nhas_ace = []\nfor i in player_card_feature:\n    if ('A' in i[0][0:2]):\n        has_ace.append(1)\n    else:\n        has_ace.append(0)\nmodel_df['has_ace'] = has_ace\n\ndealer_card_num = []\nfor i in model_df['dealer_card']:\n    if i=='A':\n        dealer_card_num.append(11)\n    else:\n        dealer_card_num.append(i)\nmodel_df['dealer_card_num'] = dealer_card_num\n"

In [70]:
dealer_card_feature, player_card_feature, player_results

([2,
  8,
  8,
  7,
  10,
  4,
  7,
  10,
  'A',
  10,
  9,
  8,
  5,
  6,
  10,
  'A',
  2,
  9,
  6,
  8,
  5,
  10,
  6,
  10,
  10,
  10,
  8,
  10,
  9,
  10,
  4],
 [[['A', 10]],
  [[10, 5]],
  [[2, 6]],
  [[10, 10, 10]],
  [[5, 10, 9]],
  [[3, 10, 10]],
  [[7, 6, 10]],
  [[9, 4, 6]],
  [[10, 3]],
  [['A', 7, 9]],
  [[10, 2, 10]],
  [['A', 10]],
  [[5, 10, 9]],
  [[9, 3]],
  [[4, 'A']],
  [[8, 8]],
  [[6, 10]],
  [[10, 3, 8]],
  [[3, 3]],
  [[4, 8]],
  [[10, 2]],
  [[10, 7]],
  [[10, 10, 10]],
  [[8, 'A', 4]],
  [['A', 3, 10]],
  [[4, 10]],
  [[2, 5]],
  [[2, 3]],
  [[8, 7]],
  [['A', 3, 6, 7, 3, 5]],
  [[2, 4]]],
 [[1.0],
  [0.0],
  [1.0],
  [-1.0],
  [-1.0],
  [-1.0],
  [-1.0],
  [0.0],
  [1.0],
  [0.0],
  [-1.0],
  [1.0],
  [-1.0],
  [0.0],
  [1.0],
  [-1.0],
  [1.0],
  [1.0],
  [0.0],
  [1.0],
  [0.0],
  [-1.0],
  [-1.0],
  [0.0],
  [0.0],
  [1.0],
  [1.0],
  [-1.0],
  [1.0],
  [-1.0],
  [1.0]])

In [96]:
# Use this loop to create features and parse multiple decisions within one game
f_dealer_card = []
f_total_current = []
f_total_initial = []
f_decision = []
f_hasAce = []
f_numberCards = []
t_outcome = []

for game,result in enumerate(player_results):
    
    # Get outcome
    outcome = result[0]
    
    # Get dealer face-up card
    dealer_card = dealer_card_feature[game]
    
    # Process the ending hand to split it up into decision points
    ending_hand = player_card_feature[game]
    
    # Get player's initial card
    player_initial = total_up(ending_hand[0][0:2])
    
    # If len of ending hand is 2, the decision was to stay (0)
    if len(ending_hand[0]) == 2:
        
        # If the initial total is 21, the player had to make no decisions
        if player_initial < 21:
            decision = 0
            decision_hand = ending_hand[0]
            print(f'Decision: {decision}')
            print(f'Decision Hand: {decision_hand}')

            # Append features/target to lists
            t_outcome.append(outcome)
            f_dealer_card.append(dealer_card)
            f_total_initial.append(player_initial)
            f_total_current.append(total_up(decision_hand))
            f_decision.append(decision)
            f_hasAce.append('A' in decision_hand)
            f_numberCards.append(len(decision_hand))
        
    else:
        # First decision to hit
        decision = 1
        decision_hand = ending_hand[0][0:2]
        print(f'Decision: {decision}')
        print(f'Decision Hand: {decision_hand}')
        
        # Append features/target to lists
        t_outcome.append(outcome)
        f_dealer_card.append(dealer_card)
        f_total_initial.append(player_initial)
        f_total_current.append(total_up(decision_hand))
        f_decision.append(decision)
        f_hasAce.append('A' in decision_hand)
        f_numberCards.append(len(decision_hand))
        
        # Figure out subsequent decisions
        for i in range(2,len(ending_hand[0])):
            
            pot_decision_hand = ending_hand[0][0:i+1]
            
            if total_up(pot_decision_hand) < 21:
                
                # a subsequent decision was made so set potential decision hand to decision hand
                decision_hand = pot_decision_hand
                
                if (i == len(ending_hand[0])-1):
                    
                    # If we are on the last decision (and it's less than 21), it was a decision to stay
                    decision = 0
                    print(f'Decision: {decision}')
                    
                else:
                    decision = 1
                    print(f'Decision: {decision}')
                    
                print(f'Decision Hand: {decision_hand}')
                
                # Append features/target to lists
                t_outcome.append(outcome)
                f_dealer_card.append(dealer_card)
                f_total_initial.append(player_initial)
                f_total_current.append(total_up(decision_hand))
                f_decision.append(decision)
                f_hasAce.append('A' in decision_hand)
                f_numberCards.append(len(decision_hand))
            
    
    print('******')
    #for i in range(len(ending_hand[0]) - 1):
    #    print(total_up(ending_hand[0][0:2+i]))

******
Decision: 0
Decision Hand: [10, 5]
******
Decision: 0
Decision Hand: [2, 6]
******
Decision: 1
Decision Hand: [10, 10]
******
Decision: 1
Decision Hand: [5, 10]
******
Decision: 1
Decision Hand: [3, 10]
******
Decision: 1
Decision Hand: [7, 6]
******
Decision: 1
Decision Hand: [9, 4]
Decision: 0
Decision Hand: [9, 4, 6]
******
Decision: 0
Decision Hand: [10, 3]
******
Decision: 1
Decision Hand: ['A', 7]
Decision: 0
Decision Hand: ['A', 7, 9]
******
Decision: 1
Decision Hand: [10, 2]
******
******
Decision: 1
Decision Hand: [5, 10]
******
Decision: 0
Decision Hand: [9, 3]
******
Decision: 0
Decision Hand: [4, 'A']
******
Decision: 0
Decision Hand: [8, 8]
******
Decision: 0
Decision Hand: [6, 10]
******
Decision: 1
Decision Hand: [10, 3]
******
Decision: 0
Decision Hand: [3, 3]
******
Decision: 0
Decision Hand: [4, 8]
******
Decision: 0
Decision Hand: [10, 2]
******
Decision: 0
Decision Hand: [10, 7]
******
Decision: 1
Decision Hand: [10, 10]
******
Decision: 1
Decision Hand: [8, 

In [99]:
decisionDf = pd.DataFrame({'Outcome': t_outcome, 'Dealer_FaceUp': f_dealer_card,
                      'Total_Initial':f_total_initial,'Total_Current':f_total_current,
                      'Decision':f_decision,'HasAce':f_hasAce,'numCards':f_numberCards})

In [100]:
decisionDf

,Outcome,Dealer_FaceUp,Total_Initial,Total_Current,Decision,HasAce,numCards
0,0.0,8,15,15,0,False,2
1,1.0,8,8,8,0,False,2
2,-1.0,7,20,20,1,False,2
3,-1.0,10,15,15,1,False,2
4,-1.0,4,13,13,1,False,2
5,-1.0,7,13,13,1,False,2
6,0.0,10,13,13,1,False,2
7,0.0,10,13,19,0,False,3
8,1.0,A,13,13,0,False,2
9,0.0,10,18,18,1,True,2
